In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
i=0
curr = ''
with open('quartileData\\taskData_staiTop.csv', 'r' ) as theFile:
    reader = csv.reader(theFile)
    headers = next(reader, None)
    for line in reader:
        if line[70] != curr:
            if i > 0:
                block = [
                    {
                        'action': np.array([actions])-1,
                        'state': np.array([states]),
                        'reward': np.array([rewards]),
                        'id': curr,

                        'block': 0
                    }

                ]
#                 if i<226:
                data[curr] = block
#                 else:
                test_data[curr] = block
            curr = line[70]
            i += 1
            actions = []
            states = []
            rewards = []
        actions.append(int(float(line[8])))
        states.append([int(float(line[3])),int(float(line[4]))])
        rewards.append(int(float(line[10])))

In [3]:
data

{'A104V8NZIQFN2F': [{'action': array([[0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
           1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
           0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
           0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
           1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
           1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
           1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
           0, 0, 1, 1, 0]]),
   'state': array([[[ 4,  7],
           [ 7,  8],
           [ 8,  7],
           [ 7,  4],
           [ 8,  4],
           [ 7,  8],
           [ 4,  7],
           [ 4,  7],
           [ 8,  7],
           [ 7,  4],
           [ 8,  7],
           [ 4,  8],
           [ 7,  8],
           [ 4,  8],
           [ 8,  4],
           [ 4,  8],
           [ 9,  4],
           [ 4,  9],
           [ 9,  

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/taskData_staiTop/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-08-09 15:22:46,081 - DeepRL - DEBUG - version control: (None, None)
2021-08-09 15:22:46,082 - DeepRL - DEBUG - learning rate: 0.01
2021-08-09 15:22:46,082 - DeepRL - DEBUG - global iters: 50
2021-08-09 15:22:46,083 - DeepRL - DEBUG - training data-points: 257
2021-08-09 15:22:46,083 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-08-09 15:22:46,481 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/taskData_staiTop/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-09 15:23:02,374 - DeepRL - DEBUG - global iter =    0 total obj: 28321.2738
2021-08-09 15:23:18,662 - DeepRL - DEBUG - global iter =    1 total obj: 28125.7528
2021-08-09 15:23:37,422 - DeepRL - DEBUG - global iter =    2 total obj: 27619.6191
2021-08-09 15:23:54,345 - DeepRL - DEBUG - global iter =    3 total obj: 26983.3857
2021-08-09 15:24:10,899 - DeepRL - DEBUG - global iter =    4 total obj: 26519.8007
2021-08-09 15:24:28,301 - DeepRL - DEBUG - global iter =    5 total obj: 26160.3210
2021-08-09 15:24:44,973 - DeepRL - DEBUG - global iter =    6 total obj: 25876.8817
2021-08-09 15:25:01,699 - DeepRL - DEBUG - global iter =    7 total obj: 25652.2743
2021-08-09 15:25:17,537 - DeepRL - DEBUG - global iter =    8 total obj: 25552.3854
2021-08-09 15:25:34,475 - DeepRL - DEBUG - global iter =    9 total obj: 25360.0681
2021-08-09 15:25:50,364 - DeepRL - DEBUG - global iter =   10 total obj: 25202.8529
2021-08-09 15:26:06,269 - DeepRL - DEBUG - global iter =   11 total obj: 250

INFO:tensorflow:../results/taskData_staiTop/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
# from actionflow.data.data_process import DataProcess
# train_merged = DataProcess.merge_data(data)
# train_merged

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-08-09 15:36:25,926 - DeepRL - DEBUG - version control: (None, None)
2021-08-09 15:36:25,927 - DeepRL - DEBUG - learning rate: 0.01
2021-08-09 15:36:25,928 - DeepRL - DEBUG - global iters: 50
2021-08-09 15:36:25,929 - DeepRL - DEBUG - training data-points: 257
2021-08-09 15:36:25,930 - DeepRL - DEBUG - test data-points: 257
2021-08-09 15:36:26,410 - DeepRL - DEBUG - opt started...
2021-08-09 15:36:26,411 - DeepRL - DEBUG - started testing...
2021-08-09 15:37:16,251 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_staiTop/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-09 15:37:35,368 - DeepRL - DEBUG - global iter =    0 total obj: 28338.7017
2021-08-09 15:37:51,648 - DeepRL - DEBUG - global iter =    1 total obj: 28098.3183
2021-08-09 15:38:08,116 - DeepRL - DEBUG - global iter =    2 total obj: 27220.7556
2021-08-09 15:38:23,628 - DeepRL - DEBUG - global iter =    3 total obj: 26255.2958
2021-08-09 15:38:39,061 - DeepRL - DEBUG - global iter =    4 total obj: 25729.8514
2021-08-09 15:38:54,491 - DeepRL - DEBUG - global iter =    5 total obj: 25468.7761
2021-08-09 15:39:09,942 - DeepRL - DEBUG - global iter =    6 total obj: 25305.3975
2021-08-09 15:39:25,427 - DeepRL - DEBUG - global iter =    7 total obj: 25203.7457
2021-08-09 15:39:40,873 - DeepRL - DEBUG - global iter =    8 total obj: 25085.6686
2021-08-09 15:39:56,277 - DeepRL - DEBUG - global iter =    9 total obj: 24997.5427
2021-08-09 15:39:56,279 - DeepRL - DEBUG - started testing...
2021-08-09 15:41:08,076 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_staiTop/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-09 15:41:28,262 - DeepRL - DEBUG - global iter =   10 total obj: 24912.6227
2021-08-09 15:41:46,493 - DeepRL - DEBUG - global iter =   11 total obj: 24827.1557
2021-08-09 15:42:04,763 - DeepRL - DEBUG - global iter =   12 total obj: 24754.5487
2021-08-09 15:42:52,904 - DeepRL - DEBUG - global iter =   13 total obj: 24685.3929
2021-08-09 15:43:16,443 - DeepRL - DEBUG - global iter =   14 total obj: 24637.9892
2021-08-09 15:44:00,424 - DeepRL - DEBUG - global iter =   15 total obj: 24564.9918
2021-08-09 15:44:18,719 - DeepRL - DEBUG - global iter =   16 total obj: 24511.2012
2021-08-09 15:44:37,375 - DeepRL - DEBUG - global iter =   17 total obj: 24473.3384
2021-08-09 15:44:55,328 - DeepRL - DEBUG - global iter =   18 total obj: 24447.2875
2021-08-09 15:45:29,277 - DeepRL - DEBUG - global iter =   19 total obj: 24409.9088
2021-08-09 15:45:29,280 - DeepRL - DEBUG - started testing...
2021-08-09 15:47:54,801 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/taskData_staiTop/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-09 15:55:32,973 - DeepRL - DEBUG - global iter =   30 total obj: 24077.6162
2021-08-09 15:55:50,644 - DeepRL - DEBUG - global iter =   31 total obj: 24007.0706
2021-08-09 15:56:33,088 - DeepRL - DEBUG - global iter =   32 total obj: 24019.8243
2021-08-09 15:56:56,526 - DeepRL - DEBUG - global iter =   33 total obj: 23978.0414
2021-08-09 15:57:22,633 - DeepRL - DEBUG - global iter =   34 total obj: 23983.2035
2021-08-09 15:58:03,253 - DeepRL - DEBUG - global iter =   35 total obj: 23939.2608
2021-08-09 15:58:20,863 - DeepRL - DEBUG - global iter =   36 total obj: 23961.4852
2021-08-09 15:58:38,931 - DeepRL - DEBUG - global iter =   37 total obj: 23939.5568
2021-08-09 15:58:56,723 - DeepRL - DEBUG - global iter =   38 total obj: 23905.3851
2021-08-09 15:59:44,962 - DeepRL - DEBUG - global iter =   39 total obj: 23888.7726
2021-08-09 15:59:44,963 - DeepRL - DEBUG - started testing...
2021-08-09 16:02:29,570 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_staiTop/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-09 16:02:52,984 - DeepRL - DEBUG - global iter =   40 total obj: 23869.5480
2021-08-09 16:03:11,897 - DeepRL - DEBUG - global iter =   41 total obj: 23838.0477
2021-08-09 16:03:29,730 - DeepRL - DEBUG - global iter =   42 total obj: 23864.6185
2021-08-09 16:03:48,083 - DeepRL - DEBUG - global iter =   43 total obj: 23870.2635
2021-08-09 16:04:06,670 - DeepRL - DEBUG - global iter =   44 total obj: 23846.8371
2021-08-09 16:04:24,921 - DeepRL - DEBUG - global iter =   45 total obj: 23816.9579
2021-08-09 16:04:44,251 - DeepRL - DEBUG - global iter =   46 total obj: 23820.9309
2021-08-09 16:05:02,434 - DeepRL - DEBUG - global iter =   47 total obj: 23793.9539
2021-08-09 16:05:21,525 - DeepRL - DEBUG - global iter =   48 total obj: 23794.6277
2021-08-09 16:05:39,933 - DeepRL - DEBUG - global iter =   49 total obj: 23861.8504
2021-08-09 16:05:39,935 - DeepRL - DEBUG - opt finished.
2021-08-09 16:05:39,935 - DeepRL - DEBUG - started testing...
2021-08-09 16:09:18,627 - DeepRL - DEBUG 

INFO:tensorflow:../results/taskData_staiTop/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# test_data["S248"]